# Chinook music webstore
The goal of this project is to answer simple business questions regarding a fictious music marketplace called Chinook. We will use SQL language to perform the analysis.

The database containing the data is in the chinook.db and chinook-unmodified.db files.

In [1]:
%%capture
%load_ext sql
%sql sqlite:///chinook.db

'Connected: None@chinook.db'

### Get familiar with the data inside the database

In [2]:
%%sql
SELECT
    name,
    type
FROM sqlite_master
WHERE type IN ("table","view");

Done.


name,type
album,table
artist,table
customer,table
employee,table
genre,table
invoice,table
invoice_line,table
media_type,table
playlist,table
playlist_track,table


## Perform a Market analysis: most sold Music Genres in USA

Relation between tables:
* track -> (track_id, genre_id,...)
* invoice_line -> (invoice_line_id, invoice_id, track_id, unit_price, quantity)
* invoice -> (invoice_id, customer_id, ...)
* customer -> (customer_id, country, ...)
* genre -> (genre_id, genre)

To get the most sold genres in USA we must connect:

genre ->(genre_id)-> track 

track ->(track_id)-> invoice_line

track -> (track_id) -> playlist_track

playlist_track -> (playlist_id) -> playlist

invoice_line ->(invoice_id)-> invoice

invoice ->(customer_id)-> customer -> country

In [3]:
%%sql
SELECT gen.name, pl.name, SUM(invl.unit_price*invl.quantity)
FROM genre AS gen
LEFT JOIN track AS tr ON gen.genre_id = tr.genre_id
INNER JOIN invoice_line AS invl ON tr.track_id = invl.track_id
INNER JOIN playlist_track AS plt ON tr.track_id = plt.track_id
INNER JOIN playlist AS pl ON plt.playlist_id = pl.playlist_id
INNER JOIN invoice AS inv ON invl.invoice_id = inv.invoice_id
INNER JOIN customer AS c ON inv.customer_id = c.customer_id
WHERE (pl.name LIKE "%Music%"
       AND c.country == 'USA')
GROUP BY 1
ORDER BY 3 DESC
LIMIT 3

Done.


name,name_1,SUM(invl.unit_price*invl.quantity)
Rock,Music,1359.270000000011
Alternative & Punk,Music,287.10000000000116
Metal,Music,282.1500000000011


The 3 Most Sold Genres are Rock, Alternative & Punk and Metal in that given order.

### Asessing sale performance of employees

Additional factors that might affect the employee sales:
* Geographical distribution. Sales in some countries might be stronger than others.
* Some employees might have been incorporated to the team later.

In [4]:
%%sql
SELECT 
    emp.first_name || " " || emp.last_name AS employee_name,
    SUM(inv.total) AS total_sold,
    AVG(inv.total) AS avg_sold,
    COUNT(inv.total) As number_sales,
    MIN(inv.invoice_date) AS first_transaction,
    MAX(inv.invoice_date) AS last_transaction
FROM invoice AS inv
LEFT JOIN customer AS c ON inv.customer_id = c.customer_id
LEFT JOIN employee AS emp ON c.support_rep_id = emp.employee_id
GROUP BY 1, emp.employee_id

Done.


employee_name,total_sold,avg_sold,number_sales,first_transaction,last_transaction
Jane Peacock,1731.5100000000039,8.167500000000018,212,2017-01-03 00:00:00,2020-12-30 00:00:00
Margaret Park,1584.0000000000034,7.401869158878521,214,2017-01-05 00:00:00,2020-12-29 00:00:00
Steve Johnson,1393.920000000002,7.414468085106393,188,2017-01-10 00:00:00,2020-12-20 00:00:00


The Sales Agent who sold most was Jane Peacock. Seems that the data for the agents covers the identical same time span. Moreover the distribution of countries where they sell seems similar. Jane seems to have done a similar number of sales to the other agents, however, the average sell is around 10% higher than the ones of the other agents.

### Analysis of sales per country

In [5]:
%%sql
DROP VIEW sales_country_ordered;
CREATE VIEW sales_country_ordered AS
    WITH 
    sales_country AS (
        SELECT 
            c.first_name || " " || c.last_name AS customer_name,
            c.country,
            inv.total AS sale
        FROM invoice_line AS invl
        LEFT JOIN invoice AS inv ON invl.invoice_id = inv.invoice_id
        LEFT JOIN customer AS c ON inv.customer_id = c.customer_id
    ),
    others_countries AS (
        SELECT
            CASE
            WHEN COUNT(customer_name) <= 100 THEN country
            END
        FROM sales_country
        GROUP BY country
    ),
    sales_country_and_others AS(
        SELECT
            CASE
                WHEN country in (SELECT * FROM others_countries) THEN "Others" 
                ELSE country
            END AS customer_country,
            *
        FROM sales_country
    )

    SELECT 
        customer_country,
        COUNT(customer_name) AS number_customers,
        SUM(sale) AS total_value_sales,
        SUM(sale)/COUNT(customer_name) AS average_sales_customer,
        AVG(sale) AS average_order,
        CASE
        WHEN customer_country == "Others" THEN 0
        ELSE 1
        END AS orderflag
    FROM sales_country_and_others
    GROUP BY customer_country
    ORDER BY orderflag DESC


Done.
Done.


[]

In [6]:
%%sql
SELECT *
FROM sales_country_ordered

Done.


customer_country,number_customers,total_value_sales,average_sales_customer,average_order,orderflag
Brazil,432,4058.999999999997,9.395833333333327,9.395833333333327,1
Canada,541,5489.549999999994,10.147042513863205,10.147042513863205,1
Czech Republic,276,3183.8400000000074,11.53565217391307,11.53565217391307,1
France,393,3972.869999999995,10.109083969465637,10.109083969465637,1
Germany,338,3441.2399999999925,10.18118343195264,10.18118343195264,1
India,185,1887.9299999999994,10.205027027027024,10.205027027027024,1
Ireland,116,1433.5199999999986,12.357931034482746,12.357931034482746,1
Portugal,187,1755.2700000000011,9.3864705882353,9.3864705882353,1
USA,1051,10405.889999999912,9.900941960037976,9.900941960037976,1
United Kingdom,248,2498.7600000000025,10.075645161290332,10.075645161290332,1


I did not find any country with 1 or less customers, however I grouped the coutnries with 100 or less customers into the Others group. I found out that Denmark has a remarkably smaller average sale price than other countries, almost half.

### Find out the percentage of single to full-album purchases

We must consider explicitly the following edge cases:
* Ignore albums with only few tracks (<=2)
* Customers might add manually every track in the album and add some others. Apparently, this does not happen often.

Attack plan for this problem:
* Figure out all the possible albums, this is not necessary, since we assume that all the tracks are from the same album when an album purchase takes place.
* Compare the track list to the track list of the album with EXCEPT. If equal add album_purchase = 1 flag.
* Create a table of album songs for each album
* Create a table of songs in each invoice

In [45]:
%%sql

DROP VIEW album_sales;
CREATE VIEW album_sales AS
    SELECT 
        tr.track_id,
        tr.album_id,
        inv.total AS sale,
        inv.invoice_id
    FROM invoice_line AS invl
    LEFT JOIN invoice AS inv ON invl.invoice_id = inv.invoice_id
    LEFT JOIN track AS tr ON invl.track_id = tr.track_id;

WITH invoice_album AS (
    SELECT
        al1.invoice_id,
        CASE
        WHEN 
            (
            SELECT al2.track_id 
            FROM album_sales AS al2 
            WHERE al2.invoice_id == al1.invoice_id
            
            EXCEPT
    
            SELECT al2.track_id 
            FROM album_sales AS al2 
            WHERE al2.album_id == al1.album_id
            GROUP BY al2.track_id
            ) IS NULL
            AND
            (
            SELECT al2.track_id 
            FROM album_sales AS al2 
            WHERE al2.album_id == al1.album_id
            GROUP BY al2.track_id
            
            EXCEPT
                
            SELECT al2.track_id 
            FROM album_sales AS al2 
            WHERE al2.invoice_id == al1.invoice_id
            ) IS NULL THEN 1
        ELSE 0
        END AS is_album
    FROM album_sales AS al1
)

SELECT 
    is_album,
    COUNT(invoice_id) AS number_invoices, 
    CAST(COUNT(invoice_id) AS Float)/(SELECT COUNT(*) 
                                      FROM invoice_album
                                     )*100 AS percentage_invoices 
FROM invoice_album
GROUP BY is_album


Done.
Done.
Done.


is_album,number_invoices,percentage_invoices
0,3281,0.6897204120243852
1,1476,0.3102795879756149


We see that about the 31% of the songs purchased are done in an album fashion. Since this number is rather high, it might make sense to not buy single tracks from the record companies

### Other business questions
* Which artist is used in most playlists?
* Purchased vs Not Purchased tracks percentage
* Is the range of tracks reflective of their popularity?
* Protected vs not ptotected have an effect in popularity?

In [53]:
%%sql
WITH artist_in_playlists AS (
    SELECT 
        art.name,
        plt.playlist_id
    FROM invoice_line AS invl
    INNER JOIN invoice AS inv ON invl.invoice_id = inv.invoice_id
    INNER JOIN track AS tr ON invl.track_id = tr.track_id
    INNER JOIN playlist_track AS plt ON tr.track_id = plt.track_id
    INNER JOIN album AS al ON tr.album_id = al.album_id
    INNER JOIN artist AS art ON al.artist_id = art.artist_id
    GROUP BY art.name, art.artist_id, plt.playlist_id
)

SELECT 
    name, 
    COUNT(playlist_id) AS number_playlists
FROM artist_in_playlists
GROUP BY name
ORDER BY number_playlists DESC
LIMIT 10

Done.


name,number_playlists
Academy of St. Martin in the Fields & Sir Neville Marriner,5
Gustav Mahler,5
James Levine,5
Luciano Pavarotti,5
Maurizio Pollini,5
Scholars Baroque Ensemble,5
The King's Singers,5
Wilhelm Kempff,5
Yo-Yo Ma,5
Accept,4


The most used artists in playlists are Academy of St. Martin in the Fields & Sir Neville Marriner, Gustav Mahler and others. They do not seem very famous.

In [75]:
%%sql
WITH 
purchased_tracks AS (
    SELECT track_id FROM invoice_line
    GROUP BY track_id
),
not_purchased_tracks AS (
    SELECT track_id FROM track
    EXCEPT
    SELECT track_id FROM purchased_tracks
)

SELECT 
    (SELECT COUNT(*) FROM purchased_tracks) AS number_purchased,
    (SELECT COUNT(*) FROM not_purchased_tracks) AS number_not_purchased

Done.


number_purchased,number_not_purchased
1806,1697


About there are as many purchased songs as not purchased. This means that Chinook should consider not buying a large amount of songs from the catallog to the record companies.

In [108]:
%%sql
WITH 
purchased_track_ranking AS (
    SELECT 
        track_id,
        COUNT(track_id) AS number_sold  
    FROM invoice_line
    GROUP BY track_id
    ORDER BY track_id DESC
), 
top100 AS (
    SELECT track_id, number_sold FROM purchased_track_ranking
    LIMIT 100
), 
top500 AS (
    SELECT track_id, number_sold FROM purchased_track_ranking
    LIMIT 500
)

SELECT 
    CAST((SELECT SUM(number_sold) 
          FROM top100
         ) AS Float)/(SELECT SUM(number_sold) 
                      FROM purchased_track_ranking) AS perc_top100,
    CAST((SELECT SUM(number_sold) 
          FROM top500
         ) AS Float)/(SELECT SUM(number_sold) 
                      FROM purchased_track_ranking) AS perc_top500,
    (SELECT COUNT(*) FROM track) AS number_tracks

Done.


perc_top100,perc_top500,number_tracks
0.0636956064746689,0.2884170695816691,3503


In [111]:
%%sql
WITH 
purchased_track_ranking AS (
    SELECT 
        track_id,
        COUNT(track_id) AS number_sold  
    FROM invoice_line
    GROUP BY track_id
    ORDER BY number_sold DESC
),
test AS (
    SELECT * FROM purchased_track_ranking
)


SELECT * FROM purchased_track_ranking LIMIT 10

Done.


track_id,number_sold
3336,31
1489,14
1495,14
6,13
1487,13
1490,13
1483,12
2558,12
1129,11
1479,11


The Top100 and Top500 Most Sold songs account for 6% and 29% of the sales, however they account for 5.5% and 28% of the songs ever purchased respectively. Therefore the distribution of Sold songs seems rather uniform. Of course, about half the catalogue was never sold, which should be taken into account.

## Conclusion
In this project we learnt the following:
* Most profittable/sold genre in USA is Rock by far, and it generates over 4 times more revenue than the following genres, Alternative&Punk or MEtal.
* USA is the biggest market. The sales per customer are very similar in most countries, with some outliers such as Denmark, where the sales per customer are half.
* About 30% of the songs are sold by album purchase.
* The Top100 and Top500 Most Sold songs account for 6% and 29% of the sales